In [15]:
import pandas as pd
# Load the dataset
data = pd.read_csv("Allergen_Status_of_Food_Products.csv")

# Display the first few rows of the dataset for exploration
data.head()


,Food Product,Main Ingredient,Sweetener,Fat/Oil,Seasoning,Allergens,Price ($),Customer rating (Out of 5),Prediction
0,Almond Cookies,Almonds,Sugar,Butter,Flour,"Almonds, Wheat, Dairy",10.15,3.1,Contains
1,Almond Cookies,Almonds,Sugar,Butter,Flour,"Almonds, Wheat, Dairy",6.17,4.5,Contains
2,Chicken Noodle Soup,Chicken broth,NaN,NaN,Salt,"Chicken, Wheat, Celery",19.65,4.1,Contains
3,Chicken Noodle Soup,Chicken broth,NaN,NaN,Salt,"Chicken, Wheat, Celery",17.48,4.7,Contains
4,Cheddar Cheese,Cheese,NaN,NaN,Salt,Dairy,10.83,3.7,Contains


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import load_model

In [6]:

# Preprocessing the data
data = pd.read_csv("Allergen_Status_of_Food_Products.csv")

# Fill NaN values with empty strings in the relevant columns
data = data.fillna('')

# Encoding the input features using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Food Product'] + " " + 
                             data['Main Ingredient'] + " " + 
                             data['Sweetener'] + " " + 
                             data['Fat/Oil'] + " " + 
                             data['Seasoning'])

# Preparing the target variable for multi-label classification
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(data['Allergens'].str.split(', '))

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [29]:

# Create the model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
# Convert the sparse matrix to a dense NumPy array
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

# Train the model with the dense data
history = model.fit(X_train_dense, y_train, epochs=800, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set using the dense data
loss, accuracy = model.evaluate(X_test_dense, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")

# Make predictions using the dense data
predictions = model.predict(X_test_dense)
model.save('ALE.h5')  # creates a HDF5 file 'my_model.h5'




Epoch 1/800
8/8 [==============================] - 0s 10ms/step - loss: 0.6858 - accuracy: 0.0510 - val_loss: 0.6452 - val_accuracy: 0.0625
Epoch 2/800
8/8 [==============================] - 0s 3ms/step - loss: 0.6245 - accuracy: 0.1059 - val_loss: 0.5815 - val_accuracy: 0.2344
Epoch 3/800
8/8 [==============================] - 0s 3ms/step - loss: 0.5566 - accuracy: 0.1294 - val_loss: 0.4976 - val_accuracy: 0.3750
Epoch 4/800
8/8 [==============================] - 0s 3ms/step - loss: 0.4734 - accuracy: 0.2275 - val_loss: 0.3919 - val_accuracy: 0.5156
Epoch 5/800
8/8 [==============================] - 0s 3ms/step - loss: 0.3730 - accuracy: 0.2471 - val_loss: 0.2831 - val_accuracy: 0.5156
Epoch 6/800
8/8 [==============================] - 0s 3ms/step - loss: 0.2863 - accuracy: 0.2549 - val_loss: 0.2020 - val_accuracy: 0.6094
Epoch 7/800
8/8 [==============================] - 0s 3ms/step - loss: 0.2235 - accuracy: 0.3294 - val_loss: 0.1603 - val_accuracy: 0.6719
Epoch 8/800
8/8 [=========

In [4]:
trained_model = load_model('ALE.h5')

In [8]:
def predict_allergens_with_user_input(model, vectorizer, mlb, threshold=0.5):
    """
    Predicts allergens in a given food product based on user inputted ingredients
    and provides the likelihood of each allergen.

    Parameters:
    - model: Trained machine learning model for prediction.
    - vectorizer: CountVectorizer fitted on the training data.
    - mlb: MultiLabelBinarizer fitted on the training data.
    - threshold: Threshold for predicting the presence of an allergen (default is 0.5).

    Returns:
    - A dictionary with allergens and their likelihood.
    """

    # User input
    food_product = input("Enter Food Product Name: ")
    main_ingredient = input("Enter Main Ingredient: ")
    sweetener = input("Enter Sweetener (or None): ")
    fat_oil = input("Enter Fat/Oil (or None): ")
    seasoning = input("Enter Seasoning (or None): ")

    # Combining the input data and transforming it using the vectorizer
    combined_input = vectorizer.transform([f"{food_product} {main_ingredient} {sweetener} {fat_oil} {seasoning}"])

    # Getting model predictions
    pred_probabilities = model.predict(combined_input.toarray())[0]

    # Creating a dictionary of allergen probabilities
    allergen_probabilities = {allergen: prob for allergen, prob in zip(mlb.classes_, pred_probabilities)}

    # Filtering to include only allergens with probability above the threshold
    likely_allergens = {allergen: prob for allergen, prob in allergen_probabilities.items() if prob > threshold}
    if likely_allergens:
        return likely_allergens
    else:
        return "Does not contain allergens"

allergen_predictions = predict_allergens_with_user_input(trained_model, vectorizer, mlb)
allergen_predictions


1/1 [==============================] - 0s 15ms/step


{'Cocoa': 0.9200398, 'Dairy': 0.9999988, 'Wheat': 0.9997657}